In [222]:
import pandas as pd
import numpy as np

from statistics import median, mean

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import OneHotEncoder

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem import PorterStemmer, SnowballStemmer

pd.set_option('display.max_rows', 90000)
pd.set_option('display.max_columns', 50)

path = '/Users/nikhilsawal/OneDrive/machine_learning/data_science_case_studies/buildzoom/data/'

In [195]:
X_train = pd.read_table(path + 'train_data.csv')
X_test = pd.read_table(path + 'xtest_data.csv')
y_test = pd.read_csv(path + 'ytest_pred.csv')

In [141]:
X_train.head()

,licensetype,businessname,legaldescription,description,type,subtype,job_value
0,NaN,ADT LLC,NaN,INSTALL LOW VOLTAGE SECURITY SYSTEM,ELECTRICAL,COMMERCIAL,NaN
1,NaN,NaN,NaN,INSTALL (1) NON-ILLUMINATED WALL SIGN - PLAZA ...,SIGN/BILLBOARD,COMMERCIAL,NaN
2,SPECIALTY CONTRACTOR LICENSE,KLN MEDIA LLC,NaN,INSTALL (1) NON-ILLUM ON-PREMISES WALL SIGN FO...,SIGN/BILLBOARD,COMMERCIAL,NaN
3,"ELECTRICAL CONTRACTOR LICENSE, ELECTRICAL CONT...",OLSON ENERGY SERVICE,NaN,REPLACE OIL FURNACE W/ NEW GAS FURNACE,MECHANICAL /,SINGLE FAMILY / DUPLEX,NaN
4,NaN,NaN,NaN,WIRE NEW SINGLE FAMILY RESIDENCE W/ 200 AMP SE...,ELECTRICAL,SINGLE FAMILY / DUPLEX,NaN


In [142]:
X_train.dtypes

licensetype         object
businessname        object
legaldescription    object
description         object
type                object
subtype             object
job_value           object
dtype: object

# License Type

In [143]:
print("Count of Uniques: ", len(X_train['licensetype'].unique()))
print("20 Unique values: ", X_train['licensetype'].unique()[:20])


Count of Uniques:  121
20 Unique values:  [nan 'SPECIALTY CONTRACTOR LICENSE'
 'ELECTRICAL CONTRACTOR LICENSE, ELECTRICAL CONTRACTOR LICENSE, REFRIGERATION CONTRACTOR LIC., GENERAL'
 'GENERAL CONTRACTOR LICENSE, REGIS. SIDE SEWER CONTRACTOR, GENERAL CONTRACTOR LICENSE'
 'ELEVATOR CONTRACTOR LICENSE' 'ELECTRICAL CONTRACTOR LICENSE'
 'GENERAL CONTRACTOR LICENSE, REFRIGERATION CONTRACTOR LIC.'
 'ELECTRICAL CONTRACTOR LICENSE, REFRIGERATION CONTRACTOR LIC.'
 'GENERAL CONTRACTOR LICENSE'
 'ELECTRICAL CONTRACTOR LICENSE, GENERAL CONTRACTOR LICENSE'
 'REFRIGERATION CONTRACTOR LIC., ELECTRICAL CONTRACTOR LICENSE, GENERAL CONTRACTOR LICENSE'
 'GENERAL CONTRACTOR LICENSE, GENERAL CONTRACTOR LICENSE, REGIS. SIDE SEWER CONTRACTOR'
 'GENERAL CONTRACTOR LICENSE, REGIS. SIDE SEWER CONTRACTOR'
 'ELECTRICAL CONTRACTOR LICENSE, REFRIGERATION CONTRACTOR LIC., REGIS. SIDE SEWER CONTRACTOR'
 'REFRIGERATION CONTRACTOR LIC., GENERAL CONTRACTOR LICENSE, ELECTRICAL CONTRACTOR LICENSE'
 'ELECTRICAL CONTRACTOR L

**NOTE:** From the output above we see that there are **121** unique values for `licensetype`. Also, each row is a combination of different licensetypes. For ex: the third row contains 4 licenses namely `ELECTRICAL CONTRACTOR LICENSE, ELECTRICAL CONTRACTOR LICENSE, REFRIGERATION CONTRACTOR LIC., GENERAL` where `ELECTRICAL CONTRACTOR LICENSE` is repeated twice, which just seems repeatative and needs to be cleaned up!!! 

**ALSO NOTE:** The license type `GENERAL CONTRACTOR LICENSE` appears in different formats like `GENERAL`, `GENERAL CONTRACTOR LICENSE` and so on. So, we are going to write a function `get_pattern()` that identifies these patterns and replaces them with the correct string, using another function `clean_license()`. In the example above, if we see `GENERAL`, we would want to replace it with `GENERAL CONTRACTOR LICENSE`. 

In [144]:
import re

def get_uniques(df, col_name):
    
    uniques = []
    for i in df[col_name]:
        uniques += i
    return list(set(uniques))


def get_pattern(df):
    
    uniques = get_uniques(df, 'licensetype')
    unique = []
    for i in range(len(uniques)):
        
        pattern = re.compile('^'+uniques[i][:5]+'*')
        matches = []
        for index, license in enumerate(uniques):

            if pattern.search(license) is not None:
                matches.append(license)
        
        if matches not in unique:
            unique.append(matches)
        pass
    
    licenseList = []
    licenseDict = {}
    
    for licenses in unique:
        licenseList.append(licenses)
        licenseDict[max(licenses)] = licenses
    
    return licenseList

In [145]:
# license type
X_train.loc[:,'licensetype'] = X_train.loc[:,'licensetype'].fillna('None')
X_train.loc[:,'licensetype'] = X_train.loc[:,'licensetype'].apply(lambda x: x.split(', ')).apply(lambda x: list(set(x)))   
pattern = get_pattern(X_train)
print(pattern[0])


['GENERAL CO', 'GENERAL C', 'GENERA', 'GENERAL CONTRACTOR LICENSE', 'GENERAL ', 'GENERAL CONT', 'GENERAL CONTRA', 'GENERAL']


The above output shows the different ways in which `GENERAL CONTRACTOR LICENSE` appears in `licensetype`. We will write a function `clean_license()` that replaces any other appearance like `'GENERAL CO', 'GENERAL C', 'GENERA'` with the complete license name `GENERAL CONTRACTOR LICENSE`.

In [146]:
# Clean licensetype
def clean_license(inp_list, reference):
    
    temp_list = []
    for i in inp_list:
        if i == 'None':
            temp_list.append(i)
        else:
            for j in reference:
                temp = []
                if i in j:
                    temp_list.append(max(j).lower().replace(" ", "_"))
                    break
    return temp_list



In [147]:
cleaned_license = [clean_license(item, pattern) for item in X_train['licensetype']]
print("Original Format: ", X_train['licensetype'][3],"\n")
print("Cleaned Format: ", cleaned_license[3])


Original Format:  ['ELECTRICAL CONTRACTOR LICENSE', 'GENERAL', 'REFRIGERATION CONTRACTOR LIC.'] 

Cleaned Format:  ['electrical_contractor_license', 'general_contractor_license', 'refrigeration_contractor_lic.']


From the output above we can now see that `GENERAL` is replaced with `general_contractor_license`. Futher we will sort each of these lists in alphabetical order and concatenate each elements of the list which should reduce the number of uniques and make `OneHotEncoding` easy for our ML model to ingest

In [148]:
cleaned_license = ['-'.join(sorted(i)) for i in cleaned_license]
print("Formated String: ", cleaned_license[3])
X_train.loc[:,'licensetype'] = cleaned_license


Formated String:  electrical_contractor_license-general_contractor_license-refrigeration_contractor_lic.


In [149]:
print("Unique LicenseTypes: ", len(X_train['licensetype'].unique()))

Unique LicenseTypes:  48


**NOTE:** We have considerabely reduced the number of unique licensetypes from `121` to `48`.

# Business Name

In [196]:
print("Count of Uniques: ", len(X_train['businessname'].unique()))
print("", X_train['businessname'].unique())


Count of Uniques:  17398
 ['ADT LLC' nan 'KLN MEDIA LLC' ... 'KRISTINE LOGAN'
 'WESCO CONSTRUCTION DIVISION' 'FRED STAUSS']


In [197]:
def get_businessname(data, n):
    """Set top N businessnames as factor"""
    data['businessname'].fillna('None', inplace=True)
    temp = data['businessname'].value_counts().head(n).index.values
    top_n = [i.lower().replace(" ","_") if i in temp else 'Other' for i in data['businessname']]
    return top_n

In [199]:
X_train.loc[:,'businessname'] = get_businessname(X_train, 100)
print("Uniques: ", X_train['businessname'].unique())
print("Uniques: ", len(X_train['businessname'].unique()))


Uniques:  ['adt_llc' 'none' 'Other' 'olson_energy_service'
 'metropolitan_sewer_service_llc' 'thyssenkrupp_elevator_corp'
 'merit_mechanical_inc' 'boyer_electric_company_inc' 'kone_inc'
 'blue_flame_llc' 'guardian_security_systems_inc'
 'prime_electric_incorporated' 'active_engineering'
 'reed_wright_htg_&_elec_co_inc' 'roto_rooter_services_company'
 'titan_electric_inc' 'valley_elec_co_of_mtvernon_inc'
 'evergreen_refrigeration_llc' "o'neill_plumbing_co"
 "mr_rooter_plumbing_(sposari's)" 'kemly_electric_incorporated'
 'otis_elevator_company' 'rescue_rooter_llc' 'schindler_elevator_company'
 'north_star_electric_inc' 'sewer_friendly_llc'
 'jim_dandy_sewer_services_inc' 'cardinal_heating_&_a/c_inc'
 'adt_security_services_inc' 'allied_fire_&_security' 's_e_s_incorporated'
 'c_&_r_electric_incorporated' 'pride_electric_incorporated' 'e_h_s_llc'
 's_m_e_incorporated_of_seattle' 'm_m_comfort_systems'
 'aces_four_construction_company' 'select_air_service'
 'best_plumbing_group_llc' 'bowie_e

# Legal Description

In [201]:
print("Count of Uniques: ", len(X_train['legaldescription'].unique()))
print("Uniques: ", X_train['legaldescription'].unique())


Count of Uniques:  14388
 [nan
 'LOT 3 TOGETHER WITH THE NORTH HALF OF LOT 4, BLOCK 17, LAWS 2ND ADDN'
 'MARKET PLACE QUEEN ANNE CONDO, VOL 17, PAGE 8' ...
 'PARCEL A, LBA #3005752'
 "SLY 30' OF LOT 16, ALL OF LOTS 17 & 18 & NE 5.2' OF LOT 19, BLK 12, EAST PARK ADDITION"
 'W 53 FT OF S 1/2, TR 2, BARTOS ACRE TRACTS']


Looking at the legal descriptions, we can make out that they are not very informative when it comes to making predictions. So we might just turn it into a boolean `has_LD`. 

In [205]:
X_train.loc[:,'legaldescription'] = X_train['legaldescription'].fillna('None')
X_train.loc[:,'has_ld'] = [1 if i!='None' else 0 for i in X_train['legaldescription']]


# Description

In [207]:
print("Count of Uniques: ", len(X_train['description'].unique()))
print("Uniques: ", X_train['description'].unique()[:100])


Count of Uniques:  80143
Uniques:  ['INSTALL LOW VOLTAGE SECURITY SYSTEM'
 'INSTALL (1) NON-ILLUMINATED WALL SIGN - PLAZA GARIBALDI'
 'INSTALL (1) NON-ILLUM ON-PREMISES WALL SIGN FOR &quot;PLAZA GARIBALDI&quot;'
 'REPLACE OIL FURNACE W/ NEW GAS FURNACE'
 'WIRE NEW SINGLE FAMILY RESIDENCE W/ 200 AMP SERVICE' 'REPAIR'
 'Construct addition and alterations to existing single family residence, per plan.'
 'ADDING CELLGUARD' 'Rebuild Letter' 'CAB INTERIOR UPGRADES'
 '[TEMPORARY POWER. ]' 'Cancel per customer log 11-469'
 'Expansion of existing minor communication facility consisting of removal and replacement of antennas and rooftop screening per plan.'
 'Demo existing single family dwelling.' 'PREWIRE SECURITY SYSTEM'
 'INSTALL TWO NEW 15 AMP CIRCUITS. REPLACE UNGROUNDED WIRING IN CRAWLSPACE.'
 'NEW 200 AMP SERVICE' 'INSTALL HEAT PUMP TO SERVE BEDROOM & ATTIC'
 'REROUTE SEWER & REPLACE DOWNSPOUT' 'Cancel per customer log 11-177'
 'INSTALLING (2) DUCT SMOKE DETECTORS W/AUDIBLE TEST STATIONS,

Looking at some of the descriptions we can immediately make out that some of the terms may be specific to ELECTRICAL type and we might want to identify a way to account for these terms. We will be suing NLTK to compute the TF-IDF for these descriptions.

In [208]:
# Remove stopwords, non alphabetic characters


def nltk_description(data):
    
    # Split data to analyze patterns in Electrical type vs non
    stop_words = set(stopwords.words("english"))
    ps = PorterStemmer()
    num_pattern = re.compile(r'\s*[\W0-9\s]\s*')

    # Fill NaN in licensetype
    data.loc[:,'description'] = data.loc[:,'description'].fillna('None')

    sample = []

    for index, description in enumerate(data["description"]):
        
        words = word_tokenize(description)
        no_stops = [i for i in words if i.lower() not in stop_words]
        no_special_char = [ps.stem(num_pattern.sub("",i)) for i in no_stops if ps.stem(num_pattern.sub("",i)) != '']
        descrip = " ".join(i for i in no_special_char)
        sample.append(descrip)
        
    cv = TfidfVectorizer(min_df=1, stop_words='english')
    x_traincv = cv.fit_transform(sample)
    
    return x_traincv.toarray().sum(axis=1).reshape(data.shape[0])


In [212]:
X_train.loc[:,'tf-idf_description'] = nltk_description(X_train)
print(X_train['tf-idf_description'])

0         1.944464
1         2.240490
2         2.620758
3         2.083272
4         2.602990
            ...   
100151    2.219222
100152    2.822032
100153    3.397444
100154    2.879694
100155    1.727000
Name: tf-idf_description, Length: 100156, dtype: float64


# Type

In [ ]:
print("Count of Uniques: ", len(X_train['type'].unique()))
print("Uniques: ", X_train['type'].unique())


# Subtype

In [213]:
print("Count of Uniques: ", len(X_train['subtype'].unique()))
print("Uniques: ", X_train['subtype'].unique())


Count of Uniques:  6
Uniques:  ['COMMERCIAL' 'SINGLE FAMILY / DUPLEX' nan 'MULTIFAMILY' 'INDUSTRIAL'
 'INSTITUTIONAL']


In [225]:
X_train['subtype'] = X_train['subtype'].fillna('None')
[i.lower().replace(" ", "_") for i in X_train['subtype']]

['commercial',
 'commercial',
 'commercial',
 'single_family_/_duplex',
 'single_family_/_duplex',
 'single_family_/_duplex',
 'single_family_/_duplex',
 'single_family_/_duplex',
 'none',
 'none',
 'single_family_/_duplex',
 'commercial',
 'commercial',
 'single_family_/_duplex',
 'single_family_/_duplex',
 'single_family_/_duplex',
 'single_family_/_duplex',
 'single_family_/_duplex',
 'single_family_/_duplex',
 'multifamily',
 'commercial',
 'commercial',
 'commercial',
 'commercial',
 'single_family_/_duplex',
 'multifamily',
 'multifamily',
 'commercial',
 'commercial',
 'single_family_/_duplex',
 'single_family_/_duplex',
 'commercial',
 'single_family_/_duplex',
 'single_family_/_duplex',
 'single_family_/_duplex',
 'single_family_/_duplex',
 'single_family_/_duplex',
 'single_family_/_duplex',
 'single_family_/_duplex',
 'single_family_/_duplex',
 'single_family_/_duplex',
 'single_family_/_duplex',
 'single_family_/_duplex',
 'single_family_/_duplex',
 'single_family_/_duplex'

# Job value

In [214]:
print("Count of Uniques: ", len(X_train['job_value'].unique()))
print("Uniques: ", X_train['job_value'].unique())


Count of Uniques:  5132
Uniques:  [nan '$250,000.00' '$45,000.00' ... '$321,743.00' '$164,848.00'
 '$732,408.00']


In [223]:
cleaned_job_value = X_train['job_value'].apply(lambda x: float(str(x).replace('$', '').replace(',','')))
X_train.loc[:,'job_value'] = cleaned_job_value
X_train.loc[:,'job_value'] = X_train['job_value'].fillna(0.0)
X_train.head(200)


,licensetype,businessname,legaldescription,description,type,subtype,job_value,has_ld,tf-idf_description
0,NaN,adt_llc,None,INSTALL LOW VOLTAGE SECURITY SYSTEM,ELECTRICAL,COMMERCIAL,0.0,0,1.944464
1,NaN,none,None,INSTALL (1) NON-ILLUMINATED WALL SIGN - PLAZA ...,SIGN/BILLBOARD,COMMERCIAL,0.0,0,2.240490
2,SPECIALTY CONTRACTOR LICENSE,Other,None,INSTALL (1) NON-ILLUM ON-PREMISES WALL SIGN FO...,SIGN/BILLBOARD,COMMERCIAL,0.0,0,2.620758
3,"ELECTRICAL CONTRACTOR LICENSE, ELECTRICAL CONT...",olson_energy_service,None,REPLACE OIL FURNACE W/ NEW GAS FURNACE,MECHANICAL /,SINGLE FAMILY / DUPLEX,0.0,0,2.083272
4,NaN,none,None,WIRE NEW SINGLE FAMILY RESIDENCE W/ 200 AMP SE...,ELECTRICAL,SINGLE FAMILY / DUPLEX,0.0,0,2.602990
5,"GENERAL CONTRACTOR LICENSE, REGIS. SIDE SEWER ...",metropolitan_sewer_service_llc,None,REPAIR,SIDE SEWER,SINGLE FAMILY / DUPLEX,0.0,0,1.000000
6,NaN,none,"LOT 3 TOGETHER WITH THE NORTH HALF OF LOT 4, B...",Construct addition and alterations to existing...,CONSTRUCTION / ADDITION OR ALTERATION,SINGLE FAMILY / DUPLEX,250000.0,1,2.799645
7,NaN,adt_llc,None,ADDING CELLGUARD,ELECTRICAL,SINGLE FAMILY / DUPLEX,0.0,0,1.377901
8,NaN,none,"MARKET PLACE QUEEN ANNE CONDO, VOL 17, PAGE 8",Rebuild Letter,LU POLICY,NaN,0.0,1,1.414076
9,ELEVATOR CONTRACTOR LICENSE,thyssenkrupp_elevator_corp,None,CAB INTERIOR UPGRADES,CONVEYANCE,NaN,0.0,0,1.669762


In [217]:
X_train.dtypes

licensetype            object
businessname           object
legaldescription       object
description            object
type                   object
subtype                object
job_value             float64
has_ld                  int64
tf-idf_description    float64
dtype: object